In [3]:
import torch_geometric
from torch_geometric.data import Data
import torch
import pandas as pd
import numpy as np
import h5py
from torch_geometric.utils import add_self_loops
from sklearn.model_selection import train_test_split

***
**SMG data inspection**

In [ ]:
PATH = '../data/components/features'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
ppi = 'CPDB'

f = h5py.File(f'{PATH}/{ppi}_multiomics.h5', 'r')
print(f.keys())

f['feature_names'][:]


<KeysViewHDF5 ['feature_names', 'features', 'features_raw', 'gene_names', 'mask_test', 'mask_train', 'mask_val', 'network', 'y_test', 'y_train', 'y_val']>


array([b'MF: KIRC', b'MF: BRCA', b'MF: READ', b'MF: PRAD', b'MF: STAD',
       b'MF: HNSC', b'MF: LUAD', b'MF: THCA', b'MF: BLCA', b'MF: ESCA',
       b'MF: LIHC', b'MF: UCEC', b'MF: COAD', b'MF: LUSC', b'MF: CESC',
       b'MF: KIRP', b'METH: KIRC', b'METH: BRCA', b'METH: READ',
       b'METH: PRAD', b'METH: STAD', b'METH: HNSC', b'METH: LUAD',
       b'METH: THCA', b'METH: BLCA', b'METH: ESCA', b'METH: LIHC',
       b'METH: UCEC', b'METH: COAD', b'METH: LUSC', b'METH: CESC',
       b'METH: KIRP', b'GE: KIRC', b'GE: BRCA', b'GE: READ', b'GE: PRAD',
       b'GE: STAD', b'GE: HNSC', b'GE: LUAD', b'GE: THCA', b'GE: BLCA',
       b'GE: ESCA', b'GE: LIHC', b'GE: UCEC', b'GE: COAD', b'GE: LUSC',
       b'GE: CESC', b'GE: KIRP', b'CNA: KIRC', b'CNA: BRCA', b'CNA: READ',
       b'CNA: PRAD', b'CNA: STAD', b'CNA: HNSC', b'CNA: LUAD',
       b'CNA: THCA', b'CNA: BLCA', b'CNA: ESCA', b'CNA: LIHC',
       b'CNA: UCEC', b'CNA: COAD', b'CNA: LUSC', b'CNA: CESC',
       b'CNA: KIRP'], dtype=object)

***
#### Add dimensions to graph

In [8]:
def load_h5_graph(PATH: str, LABEL_PATH: str, ppi: str, modalities: list = None):
    '''
    Load a multi-omics graph from an HDF5 file and create a PyTorch Geometric Data object.
    Adds node features, edge indices, labels, and splits for training, testing, and validation.
    
    Parameters:
      PATH: Path to the directory containing the HDF5 file.
      LABEL_PATH: Path to the label file.
      ppi: A string used to construct the filename.
      modalities: List of feature modalities to include. For example, if you want to include
                  only MF and GE, pass ['MF', 'GE']. If None, all features are included.
    
    Returns:
      data: A PyTorch Geometric Data object with the selected node features and splits.
    '''
    # Load the HDF5 file and extract the network and features
    f = h5py.File(f'{PATH}/{ppi}_multiomics.h5', 'r')

    # Build edge indices from the network matrix
    network = f['network'][:]
    src, dst = np.nonzero(network)
    edge_index_ppi = torch.tensor(np.vstack((src, dst)), dtype=torch.long)
    edge_type_ppi = torch.full((edge_index_ppi.size(1),), 0, dtype=torch.long)

    # Load node features and assign a node "name" attribute 
    features = f['features'][:]
    # Decode the feature names (they may come as byte strings)
    feature_names = [fn.decode('utf-8') if isinstance(fn, bytes) else str(fn) for fn in f['feature_names'][:]]
    
    # If a list of modalities is provided, filter features accordingly
    if modalities is not None:
        # Determine which columns have prefixes matching one of the modalities
        selected_indices = [
            i for i, name in enumerate(feature_names)
            if any(name.startswith(mod) for mod in modalities)
        ]
        # Slice the features to keep only the selected ones
        features = features[:, selected_indices]
        # Optionally, update the feature_names list too
        feature_names = [feature_names[i] for i in selected_indices]
    x = torch.from_numpy(features)
    num_nodes = x.size(0)
    node_name = f['gene_names'][...,-1].astype(str)

    # Retrieve gene names and create a mapping: gene name -> node index
    gene_name = f['gene_names'][...,-1].astype(str)
    gene_map = {g: i for i, g in enumerate(gene_name)}  # gene name -> node index

    # Load the labels and determine positive and negative nodes
    label_df = pd.read_csv(LABEL_PATH, sep='\t').astype(str) 
    label_symbols = label_df['symbol'].tolist()

    # Determine positive nodes: indices that appear in both the label file and gene_name list
    mask = [gene_map[g] for g in sorted(list(set(label_symbols) & set(gene_name)))]

    # Randomly select negative samples from those nodes not in the positive mask.
    np.random.seed(42)
    all_indices = set(range(len(gene_name)))
    negative_candidates = sorted(list(all_indices - set(mask)))
    neg_sample_size = min(len(mask), len(gene_name) - len(mask))
    neg_mask = np.random.choice(negative_candidates, size=neg_sample_size, replace=False).tolist()

    print("Negative mask indices:", neg_mask)

    # Create a label vector (1 for positive, 0 for negative)
    y = torch.zeros(len(gene_name), dtype=torch.float)
    y[mask] = 1
    y = y.unsqueeze(1)  # shape: [num_nodes, 1]

    # Combine positive and negative indices for the split
    final_mask = mask + neg_mask
    final_labels = y[final_mask].squeeze(1).numpy()  # converting to numpy for stratification

    # Split indices into train, test, and validation sets using stratification
    train_idx, test_idx, _, _ = train_test_split(final_mask, final_labels, test_size=0.2,
                                                    shuffle=True, stratify=final_labels, random_state=42)
    train_idx, val_idx, _, _ = train_test_split(train_idx, y[train_idx].numpy().squeeze(1),
                                                test_size=0.2, shuffle=True,
                                                stratify=y[train_idx].numpy().squeeze(1), random_state=42)

    # Create boolean masks for all nodes
    train_mask = torch.zeros(len(gene_name), dtype=torch.bool)
    test_mask = torch.zeros(len(gene_name), dtype=torch.bool)
    val_mask = torch.zeros(len(gene_name), dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True
    val_mask[val_idx] = True

    # Add self-loops to the edge_index
    #edge_index, _ = add_self_loops(edge_index_ppi, num_nodes=num_nodes)

    # Build the PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index_ppi, edge_type = edge_type_ppi, y=y)
    data.train_mask = train_mask.unsqueeze(1)  # unsqueeze if you want to mimic the original shape
    data.test_mask = test_mask.unsqueeze(1)
    data.val_mask = val_mask.unsqueeze(1)
    data.name = node_name  # optional: storing node names

    return data#, gene_map

def load_h5_graph_random_features(PATH: str, LABEL_PATH: str, ppi: str, modalities: list = None, randomize_features: bool = True):
    """
    Load a multi-omics graph from an HDF5 file and create a PyTorch Geometric Data object.
    Adds node features, edge indices, labels, and splits for training, testing, and validation.
    
    Parameters:
      PATH: Path to the directory containing the HDF5 file.
      LABEL_PATH: Path to the label file.
      ppi: A string used to construct the filename.
      modalities: List of feature modalities to include. For example, if you want to include
                  only MF and GE, pass ['MF', 'GE']. If None, all features are included.
      randomize_features: If True, ignores the actual features from the file and generates 
                          random features (using a normal distribution) of the appropriate shape.
    
    Returns:
      data: A PyTorch Geometric Data object with the selected node features (or random features),
            edge indices, labels, and data splits.
    """

    # Load the HDF5 file 
    f = h5py.File(f'{PATH}/{ppi}_multiomics.h5', 'r')

    # Build edge indices from the network matrix
    network = f['network'][:]
    src, dst = np.nonzero(network)
    edge_index_ppi = torch.tensor(np.vstack((src, dst)), dtype=torch.long)
    edge_type_ppi = torch.full((edge_index_ppi.size(1),), 0, dtype=torch.long)

    # Read and decode feature names
    feature_names = [fn.decode('utf-8') if isinstance(fn, bytes) else str(fn)
                     for fn in f['feature_names'][:]]
    
    # If modalities are specified, filter feature indices accordingly
    if modalities is not None:
        selected_indices = [
            i for i, name in enumerate(feature_names)
            if any(name.startswith(mod) for mod in modalities)
        ]
        # Update the feature_names to only include those filtered features
        feature_names = [feature_names[i] for i in selected_indices]
        num_features = len(selected_indices)
    else:
        num_features = len(feature_names)
    
    # Determine number of nodes from gene names
    node_name = f['gene_names'][...,-1].astype(str)
    num_nodes = len(node_name)
    
    # --- Create node features ---
    if randomize_features:
        # Generate a random feature matrix with shape (num_nodes, num_features)
        # Using a normal distribution
        x = torch.randn(num_nodes, num_features)
    else:
        # Load the full feature matrix from file
        features = f['features'][:]  # shape: [num_nodes, total_features]
        if modalities is not None:
            features = features[:, selected_indices]
        x = torch.from_numpy(features)
    
    # Create a mapping from gene name to node index
    gene_map = {g: i for i, g in enumerate(node_name)}
    
    # --- Load labels and determine positive/negative nodes ---
    label_df = pd.read_csv(LABEL_PATH, sep='\t').astype(str)
    label_symbols = label_df['symbol'].tolist()
    # Get positive nodes: indices present in both the label file and the gene names list
    mask = [gene_map[g] for g in sorted(list(set(label_symbols) & set(node_name)))]
    
    # Randomly select negative samples from nodes not in the positive mask.
    np.random.seed(2)
    all_indices = set(range(len(node_name)))
    negative_candidates = sorted(list(all_indices - set(mask)))
    neg_sample_size = min(len(mask), len(node_name) - len(mask))
    neg_mask = np.random.choice(negative_candidates, size=neg_sample_size, replace=False).tolist()

    print("Negative mask indices:", neg_mask)

    # Create label vector: 1 for positive, 0 for negative
    y = torch.zeros(len(node_name), dtype=torch.float)
    y[mask] = 1
    y = y.unsqueeze(1)

    # Combine positive and negative indices for stratified splits
    final_mask = mask + neg_mask
    final_labels = y[final_mask].squeeze(1).numpy()  # convert to numpy for stratification

    # Stratified train-test split
    train_idx, test_idx, _, _ = train_test_split(
        final_mask, final_labels, test_size=0.2, shuffle=True, stratify=final_labels, random_state=42
    )
    train_idx, val_idx, _, _ = train_test_split(
        train_idx, y[train_idx].numpy().squeeze(1), test_size=0.2, shuffle=True,
        stratify=y[train_idx].numpy().squeeze(1), random_state=42
    )

    # Create boolean masks for all nodes
    train_mask = torch.zeros(len(node_name), dtype=torch.bool)
    test_mask = torch.zeros(len(node_name), dtype=torch.bool)
    val_mask = torch.zeros(len(node_name), dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True
    val_mask[val_idx] = True

    # --- Build the PyTorch Geometric Data object ---
    data = Data(x=x, edge_index=edge_index_ppi, edge_type=edge_type_ppi, y=y)
    data.train_mask = train_mask.unsqueeze(1)
    data.test_mask = test_mask.unsqueeze(1)
    data.val_mask = val_mask.unsqueeze(1)
    data.name = node_name  # store node names as an attribute
    # data.feature_names = feature_names  # store the associated feature modality names

    return data

def load_h5_graph_shuffle(PATH: str, LABEL_PATH: str, ppi: str, shuffle_features: bool = False):
    '''
    Load a multi-omics graph from an HDF5 file and create a PyTorch Geometric Data object.
    Add the node features, edge indices, and labels to the object.
    Split the nodes into training, testing, and validation sets.
    Add multiple edge types from additional networks.
    
    If shuffle_features is True, both the rows (nodes) and columns (features) of the feature 
    matrix are shuffled. This breaks the association between nodes and their original features,
    serving as an ablation study on the importance of feature organization.
    '''
    # Load the HDF5 file and extract the network and features
    f = h5py.File(f'{PATH}/{ppi}_multiomics.h5', 'r')

    # Build edge indices from the network matrix
    network = f['network'][:]
    src, dst = np.nonzero(network)
    edge_index_ppi = torch.tensor(np.vstack((src, dst)), dtype=torch.long)
    edge_type_ppi = torch.full((edge_index_ppi.size(1),), 0, dtype=torch.long)

    # Load node features and assign a node "name" attribute 
    features = f['features'][:]
    if shuffle_features:
        # Set a fixed seed for reproducibility
        np.random.seed(42)
        # Shuffle rows (nodes)
        row_perm = np.random.permutation(features.shape[0])
        features = features[row_perm, :]
        # Shuffle columns (features)
        col_perm = np.random.permutation(features.shape[1])
        features = features[:, col_perm]
    x = torch.from_numpy(features)
    num_nodes = x.size(0)
    node_name = f['gene_names'][...,-1].astype(str)

    # Retrieve gene names and create a mapping: gene name -> node index
    gene_name = f['gene_names'][...,-1].astype(str)
    gene_map = {g: i for i, g in enumerate(gene_name)}  # gene name -> node index

    # Load the labels and determine positive and negative nodes
    label_df = pd.read_csv(LABEL_PATH, sep='\t').astype(str) 
    label_symbols = label_df['symbol'].tolist()

    # Determine positive nodes: indices that appear in both the label file and gene_name list
    mask = [gene_map[g] for g in sorted(list(set(label_symbols) & set(gene_name)))]

    # Randomly select negative samples from those nodes not in the positive mask.
    np.random.seed(42)
    all_indices = set(range(len(gene_name)))
    negative_candidates = sorted(list(all_indices - set(mask)))
    neg_sample_size = min(len(mask), len(gene_name) - len(mask))
    neg_mask = np.random.choice(negative_candidates, size=neg_sample_size, replace=False).tolist()

    print("Negative mask indices:", neg_mask)

    # Create a label vector (1 for positive, 0 for negative)
    y = torch.zeros(len(gene_name), dtype=torch.float)
    y[mask] = 1
    y = y.unsqueeze(1)  # shape: [num_nodes, 1]

    # Combine positive and negative indices for the split
    final_mask = mask + neg_mask
    final_labels = y[final_mask].squeeze(1).numpy()  # converting to numpy for stratification

    # Split indices into train, test, and validation sets using stratification
    train_idx, test_idx, _, _ = train_test_split(final_mask, final_labels, test_size=0.2,
                                                    shuffle=True, stratify=final_labels, random_state=42)
    train_idx, val_idx, _, _ = train_test_split(train_idx, y[train_idx].numpy().squeeze(1),
                                                test_size=0.2, shuffle=True,
                                                stratify=y[train_idx].numpy().squeeze(1), random_state=42)

    # Create boolean masks for all nodes
    train_mask = torch.zeros(len(gene_name), dtype=torch.bool)
    test_mask = torch.zeros(len(gene_name), dtype=torch.bool)
    val_mask = torch.zeros(len(gene_name), dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True
    val_mask[val_idx] = True

    # Build the PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index_ppi, edge_type=edge_type_ppi, y=y)
    data.train_mask = train_mask.unsqueeze(1)
    data.test_mask = test_mask.unsqueeze(1)
    data.val_mask = val_mask.unsqueeze(1)
    data.name = node_name  # optional: storing node names

    return data

def add_all_edges(data, NETWORK_PATH: str, NETWORKS: list):

    # Add additional edges from the other networks
    for i, network_name in enumerate(NETWORKS):
        adj_matrix = pd.read_csv(f'{NETWORK_PATH}/network_{network_name}.csv', index_col=0)
        data = add_edge_type(data, adj_matrix, i+1)

    return data

def add_edge_type(data, adj_matrix: pd.DataFrame, edge_type_label: int):
    '''
    Adds edges from an additional binary adjacency matrix to the existing PyG data object.
    
    Parameters:
    - data: PyTorch Geometric Data object (already contains data.name with gene names)
    - network: adjacency matrix of the additional edges in the form of a pandas DataFrame
    - edge_type: integer that will label all these additional edges (e.g., 2)
    
    Returns:
    - data: updated Data object with the additional edges and their edge types appended.
    '''
    additional_row, additional_col = np.nonzero(adj_matrix)
    row_names = adj_matrix.index
    col_names = adj_matrix.columns

    # Create mapping from gene name (from the first network) to its node index
    gene_to_index = {gene: idx for idx, gene in enumerate(data.name)}

    # Convert these indices to the indices in the main graph using gene_to_index.
    global_rows = []
    global_cols = []

    # not_found = []

    for r, c in zip(additional_row, additional_col):
        gene_r = row_names[r]
        gene_c = col_names[c]
        # Check that both gene names exist in the main mapping
        if gene_r in gene_to_index and gene_c in gene_to_index:
            global_rows.append(gene_to_index[gene_r])
            global_cols.append(gene_to_index[gene_c])
        else:
            # if gene_r not in gene_to_index and gene_r not in not_found:
            #     not_found.append(gene_r)
            # if gene_c not in gene_to_index and gene_c not in not_found:
            #     not_found.append(gene_c)
            #print(f"Gene {gene_r} or {gene_c} not found in the main graph.")
            continue
    
    if not global_rows:  # No matching edges found
        print("No matching edges found.")
        return data
    
    # Create the edge_index for the additional edges
    additional_edge_index = torch.tensor(np.vstack((global_rows, global_cols)), dtype=torch.long)
    
    # Create an edge attribute for these additional edges, using the provided type value.
    additional_edge_attr = torch.full((additional_edge_index.size(1),), edge_type_label, dtype=torch.long)
    
    # Combine the additional edges with the existing ones.
    # We assume that data.edge_index and data.edge_type already exist.
    data.edge_index = torch.cat([data.edge_index, additional_edge_index], dim=1)
    data.edge_type = torch.cat([data.edge_type, additional_edge_attr], dim=0)
    
    return data

In [14]:
def load_h5_graph_with_external_edges(PATH: str,
                                      LABEL_PATH: str,
                                      ppi: str,
                                      NETWORK_PATH: str,
                                      NETWORKS: list,
                                      modalities: list = None,
                                      randomize_features: bool = False):
    """
    Load a multi-omics graph from an HDF5 file and create a PyTorch Geometric Data object.
    Instead of using the base PPI network from the HDF5 file, this function initializes
    the graph with node features and labels but uses no initial edges. Then, additional
    edges are added from external CSV files located in NETWORK_PATH based on the provided
    list of NETWORKS.
    
    Parameters:
      PATH (str): Directory containing the HDF5 file.
      LABEL_PATH (str): Path to the label file.
      ppi (str): Identifier (part of the filename) for the HDF5 file.
      NETWORK_PATH (str): Directory containing the CSV files for additional networks.
      NETWORKS (list): List of network names to use for external edges (e.g.,
                       ['coexpression', 'GO', 'domain_sim', 'sequence_sim', 'pathway_co_occurrence']).
      modalities (list, optional): List of feature modality prefixes to include (e.g., ['MF', 'GE']).
                                   If None, all features are used.
      randomize_features (bool, optional): If True, the node features will be replaced by random
                                             values (sampled from a normal distribution).
    
    Returns:
      data: A PyTorch Geometric Data object with:
           - x: Node features (filtered or randomized as requested)
           - edge_index: External edge indices (after adding from CSV files)
           - edge_type: Integer edge types (one per external network; base graph has no edges)
           - y: Labels (with 1 for positive and 0 for negative)
           - train_mask, test_mask, val_mask: Train/val/test splits derived via stratification
           - name: The node (gene) names extracted from the HDF5 file.
    """
    import h5py
    import numpy as np
    import torch
    import pandas as pd
    from torch_geometric.data import Data
    from sklearn.model_selection import train_test_split

    # --- Load the HDF5 file and extract node features ---
    f = h5py.File(f'{PATH}/{ppi}_multiomics.h5', 'r')

    # Decode feature names from the HDF5 file
    feature_names = [fn.decode('utf-8') if isinstance(fn, bytes) else str(fn)
                     for fn in f['feature_names'][:]]
    if modalities is not None:
        # Filter columns based on modalities prefixes
        selected_indices = [i for i, name in enumerate(feature_names)
                            if any(name.startswith(mod) for mod in modalities)]
        feature_names = [feature_names[i] for i in selected_indices]
        num_features = len(selected_indices)
    else:
        num_features = len(feature_names)
    
    # Load gene/node names (assumed stored as the last column in the 'gene_names' dataset)
    node_names = f['gene_names'][...,-1].astype(str)
    num_nodes = len(node_names)
    
    # Decide on node feature matrix: randomize or use true features
    if randomize_features:
        x = torch.randn(num_nodes, num_features)
    else:
        features = f['features'][:]  # full feature matrix with shape (num_nodes, total_features)
        if modalities is not None:
            features = features[:, selected_indices]
        x = torch.from_numpy(features)
    
    # --- Create an empty graph structure ---
    # Initialize empty edge_index and edge_type. The dimensions are set such that edge_index is 2 x 0.
    edge_index = torch.empty((2, 0), dtype=torch.long)
    edge_type = torch.empty((0,), dtype=torch.long)
    
    # --- Create labels and splits ---
    # Build mapping from gene name (node_names) to node index.
    gene_to_index = {gene: idx for idx, gene in enumerate(node_names)}
    
    # Read labels from the LABEL_PATH file.
    label_df = pd.read_csv(LABEL_PATH, sep='\t').astype(str)
    label_symbols = label_df['symbol'].tolist()
    
    # Identify positive nodes (gene symbols common to both the label file and node_names)
    mask = [gene_to_index[g] for g in sorted(list(set(label_symbols) & set(node_names)))]
    
    # Randomly sample negative nodes (those not in mask)
    np.random.seed(42)
    all_indices = set(range(len(node_names)))
    negative_candidates = sorted(list(all_indices - set(mask)))
    neg_sample_size = min(len(mask), len(node_names) - len(mask))
    neg_mask = np.random.choice(negative_candidates, size=neg_sample_size, replace=False).tolist()
    
    print("Negative mask indices:", neg_mask)
    
    # Create label vector: 1 for positives, 0 for negatives
    y = torch.zeros(len(node_names), dtype=torch.float)
    y[mask] = 1
    y = y.unsqueeze(1)

    # Combine positive and negative samples for splits
    final_mask = mask + neg_mask
    final_labels = y[final_mask].squeeze(1).numpy()
    
    # Perform stratified splits into train, test, and validation sets
    train_idx, test_idx, _, _ = train_test_split(
        final_mask, final_labels, test_size=0.2, shuffle=True, stratify=final_labels, random_state=42
    )
    train_idx, val_idx, _, _ = train_test_split(
        train_idx, y[train_idx].numpy().squeeze(1), test_size=0.2, shuffle=True,
        stratify=y[train_idx].numpy().squeeze(1), random_state=42
    )
    
    # Create boolean masks
    train_mask = torch.zeros(len(node_names), dtype=torch.bool)
    test_mask  = torch.zeros(len(node_names), dtype=torch.bool)
    val_mask   = torch.zeros(len(node_names), dtype=torch.bool)
    train_mask[train_idx] = True
    test_mask[test_idx] = True
    val_mask[val_idx] = True

    # --- Build the PyTorch Geometric data object ---
    data = Data(x=x, edge_index=edge_index, edge_type=edge_type, y=y)
    data.train_mask = train_mask.unsqueeze(1)
    data.test_mask  = test_mask.unsqueeze(1)
    data.val_mask   = val_mask.unsqueeze(1)
    data.name = node_names  # store the gene (node) names
    # data.feature_names = feature_names

    # --- Add external edges from CSV files ---
    # This step appends all edges from the specified networks to the data object.
    data = add_all_edges(data, NETWORK_PATH, NETWORKS)
    
    return data

In [ ]:
PATH = '../data/components/features'
NETWORKS = ['coexpression', 
            'GO',
            'domain_sim',
            'sequence_sim',
            'pathway_co_occurrence']
modalities = ['GE', 'METH', 'CNA', 'MF']
NETWORK_PATH = '../data/components/networks/'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
PPIS = ['CPDB', 'IRefIndex_2015', 'IRefIndex', 'STRINGdb', 'PCNet']



In [ ]:
PATH = '../data/components/features'
NETWORKS = ['coexpression', 
            'domain_sim',
            'GO',
            'sequence_sim',
            'pathway_co_occurrence',
            ]
modalities = ['GE', 'METH', 'CNA', 'MF']
NETWORK_PATH = '../data/components/networks/'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
PPIS = ['CPDB', 'IRefIndex_2015', 'IRefIndex', 'STRINGdb', 'PCNet']

graph = load_h5_graph_with_external_edges(PATH, LABEL_PATH, 'CPDB', NETWORK_PATH, NETWORKS, modalities, randomize_features=False)
print(f"Number of nodes: {graph.num_nodes}")
print(f"Number of edges: {graph.num_edges}")
print(f"Number of features: {graph.num_features}")
print(f"Number of edge types: {graph.num_edge_types}")
torch.save(graph, f'../data/components/multidim_graph/5d/no_{ppi}_cdgps_multiomics.pt')

Negative mask indices: [8274, 1696, 5846, 3422, 13287, 570, 5477, 12486, 9643, 515, 6859, 862, 5350, 12681, 5296, 7353, 6616, 1180, 7486, 1061, 6114, 11107, 8648, 9917, 9720, 1052, 954, 6743, 7602, 6580, 9279, 3736, 3188, 1497, 5394, 3647, 2777, 3398, 2229, 6079, 9851, 10957, 8831, 3565, 5150, 1536, 12980, 9923, 3064, 66, 5979, 4855, 4345, 4406, 3720, 11869, 12606, 12064, 436, 10347, 13116, 13053, 1532, 746, 13129, 546, 2119, 8458, 13075, 6163, 10100, 5273, 1401, 13264, 7916, 12612, 5803, 11428, 6426, 9490, 4210, 6896, 5398, 3226, 4325, 9534, 11857, 13061, 7138, 10821, 9662, 4549, 5560, 171, 478, 8257, 5864, 7263, 10668, 2136, 2644, 11052, 9520, 2273, 3619, 5567, 9170, 10098, 3224, 13034, 12197, 2979, 3872, 370, 5242, 11026, 13458, 2972, 406, 8262, 9558, 6189, 6285, 13290, 643, 12652, 3144, 9077, 12018, 4328, 9, 1778, 8986, 12347, 13582, 9071, 2786, 10567, 4768, 1404, 8491, 6301, 4534, 4019, 5311, 5498, 659, 380, 6656, 10323, 4834, 10072, 4585, 4970, 2084, 2731, 9816, 1792, 12517, 7170

In [ ]:
## For feature ablations
PATH = '../data/components/features'
NETWORKS = ['coexpression', 
            'GO',
            'domain_sim',
            'sequence_sim',
            'pathway_co_occurrence']
modalities = []
NETWORK_PATH = '../data/components/networks/'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
PPIS = ['CPDB']

for ppi in PPIS:
    graph = load_h5_graph_random_features(PATH, LABEL_PATH, ppi)
    graph = add_all_edges(graph, NETWORK_PATH, NETWORKS)
    print(f"Number of nodes: {graph.num_nodes}")
    print(f"Number of edges: {graph.num_edges}")
    print(f"Number of features: {graph.num_features}")
    print(f"Number of edge types: {graph.num_edge_types}")
    torch.save(graph, f'../data/components/multidim_graph/6d/feature_ablations/{ppi}_cdgps_random_features.pt')
    

Negative mask indices: [8828, 9630, 12936, 12543, 11163, 5763, 1071, 11985, 7942, 10075, 1959, 260, 5846, 6122, 10277, 4505, 548, 2521, 5225, 6183, 11993, 2736, 5071, 368, 5227, 1208, 10428, 7839, 1217, 455, 10196, 3031, 8318, 12048, 11848, 10184, 3569, 9237, 11168, 5112, 2677, 1611, 10603, 8259, 12454, 7195, 10979, 1291, 1811, 6933, 4224, 351, 3772, 2136, 4492, 9718, 6586, 12617, 12549, 4784, 7526, 9920, 12624, 937, 8216, 453, 2384, 6486, 2198, 3578, 1781, 3298, 2433, 3059, 1720, 12622, 11620, 2212, 1390, 7047, 7596, 1646, 6450, 2166, 921, 2180, 11066, 824, 1519, 12379, 3043, 9362, 6406, 2071, 5719, 5505, 5472, 8191, 7926, 2421, 5691, 8153, 5206, 9350, 4471, 5746, 13268, 7309, 5791, 692, 11926, 1494, 13293, 12781, 2023, 4753, 5747, 2878, 4367, 2722, 764, 929, 1104, 13264, 9277, 9549, 8909, 6095, 2403, 10280, 2222, 1034, 3828, 5823, 2336, 11949, 1045, 5390, 1362, 8811, 10380, 1230, 918, 3708, 3302, 4713, 11082, 7869, 1678, 12546, 8090, 10878, 9894, 11001, 3496, 10413, 2045, 13078, 1347

In [ ]:
PATH = '../data/components/features'
NETWORKS = ['coexpression', 
            'GO',
            'domain_sim',
            'sequence_sim',
            'pathway_co_occurrence']
NETWORK_PATH = '../data/components/networks/'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
PPIS = ['CPDB']


for ppi in PPIS:
    graph = load_h5_graph(PATH, LABEL_PATH, ppi)
    graph = add_all_edges(graph, NETWORK_PATH, NETWORKS)
    print(f"Number of nodes: {graph.num_nodes}")
    print(f"Number of edges: {graph.num_edges}")
    print(f"Number of features: {graph.num_features}")
    print(f"Number of edge types: {graph.num_edge_types}")
    #torch.save(graph, f'../data/components/multidim_graph/5d/{ppi}_cdgp_multiomics.pt')

Negative mask indices: [8274, 1696, 5846, 3422, 13287, 570, 5477, 12486, 9643, 515, 6859, 862, 5350, 12681, 5296, 7353, 6616, 1180, 7486, 1061, 6114, 11107, 8648, 9917, 9720, 1052, 954, 6743, 7602, 6580, 9279, 3736, 3188, 1497, 5394, 3647, 2777, 3398, 2229, 6079, 9851, 10957, 8831, 3565, 5150, 1536, 12980, 9923, 3064, 66, 5979, 4855, 4345, 4406, 3720, 11869, 12606, 12064, 436, 10347, 13116, 13053, 1532, 746, 13129, 546, 2119, 8458, 13075, 6163, 10100, 5273, 1401, 13264, 7916, 12612, 5803, 11428, 6426, 9490, 4210, 6896, 5398, 3226, 4325, 9534, 11857, 13061, 7138, 10821, 9662, 4549, 5560, 171, 478, 8257, 5864, 7263, 10668, 2136, 2644, 11052, 9520, 2273, 3619, 5567, 9170, 10098, 3224, 13034, 12197, 2979, 3872, 370, 5242, 11026, 13458, 2972, 406, 8262, 9558, 6189, 6285, 13290, 643, 12652, 3144, 9077, 12018, 4328, 9, 1778, 8986, 12347, 13582, 9071, 2786, 10567, 4768, 1404, 8491, 6301, 4534, 4019, 5311, 5498, 659, 380, 6656, 10323, 4834, 10072, 4585, 4970, 2084, 2731, 9816, 1792, 12517, 7170

In [6]:
# Assume data is your PyTorch Geometric Data object
# Calculate the count of each edge type
edge_counts = torch.bincount(graph.edge_type)

print("Edge counts per type:", edge_counts)

Edge counts per type: tensor([504378,  34982,   8606,    208,    150,   8964])


In [123]:
graph.edge_index.shape

torch.Size([2, 557288])

In [124]:
graph.edge_type.shape

torch.Size([557288])

In [ ]:
PATH = '../data/components/features'
NETWORKS = ['coexpression', 
            'GO',
            'domain_sim',
            'pathway_co_occurrence',
            #'sequence_sim'
            ]
            
NETWORK_PATH = '../data/components/networks/'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
PPIS = ['CPDB', 'IRefIndex_2015', 'IRefIndex', 'STRINGdb', 'PCNet']

for ppi in PPIS:
    graph = load_h5_graph(PATH, LABEL_PATH, ppi)
    graph = add_all_edges(graph, NETWORK_PATH, NETWORKS)
    print(f"Number of nodes: {graph.num_nodes}")
    print(f"Number of edges: {graph.num_edges}")
    print(f"Number of features: {graph.num_features}")
    print(f"Number of edge types: {graph.num_edge_types}")
    torch.save(graph, f'../data/components/multidim_graph/5d/{ppi}_cdgp_multiomics.pt')

Negative mask indices: [8274, 1696, 5846, 3422, 13287, 570, 5477, 12486, 9643, 515, 6859, 862, 5350, 12681, 5296, 7353, 6616, 1180, 7486, 1061, 6114, 11107, 8648, 9917, 9720, 1052, 954, 6743, 7602, 6580, 9279, 3736, 3188, 1497, 5394, 3647, 2777, 3398, 2229, 6079, 9851, 10957, 8831, 3565, 5150, 1536, 12980, 9923, 3064, 66, 5979, 4855, 4345, 4406, 3720, 11869, 12606, 12064, 436, 10347, 13116, 13053, 1532, 746, 13129, 546, 2119, 8458, 13075, 6163, 10100, 5273, 1401, 13264, 7916, 12612, 5803, 11428, 6426, 9490, 4210, 6896, 5398, 3226, 4325, 9534, 11857, 13061, 7138, 10821, 9662, 4549, 5560, 171, 478, 8257, 5864, 7263, 10668, 2136, 2644, 11052, 9520, 2273, 3619, 5567, 9170, 10098, 3224, 13034, 12197, 2979, 3872, 370, 5242, 11026, 13458, 2972, 406, 8262, 9558, 6189, 6285, 13290, 643, 12652, 3144, 9077, 12018, 4328, 9, 1778, 8986, 12347, 13582, 9071, 2786, 10567, 4768, 1404, 8491, 6301, 4534, 4019, 5311, 5498, 659, 380, 6656, 10323, 4834, 10072, 4585, 4970, 2084, 2731, 9816, 1792, 12517, 7170

In [ ]:
PATH = '../data/components/features'
NETWORKS = [#'coexpression', 
            #'domain_sim',
            #'GO',
            #'pathway_co_occurrence',
            'sequence_sim'
            ]
            
NETWORK_PATH = '../data/components/networks/'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
PPIS = ['CPDB']

for ppi in PPIS:
    graph = load_h5_graph(PATH, LABEL_PATH, ppi)
    graph = add_all_edges(graph, NETWORK_PATH, NETWORKS)
    print(f"Number of nodes: {graph.num_nodes}")
    print(f"Number of edges: {graph.num_edges}")
    print(f"Number of features: {graph.num_features}")
    print(f"Number of edge types: {graph.num_edge_types}")
    torch.save(graph, f'../data/components/multidim_graph/2d/{ppi}_s_multiomics.pt')

Negative mask indices: [8274, 1696, 5846, 3422, 13287, 570, 5477, 12486, 9643, 515, 6859, 862, 5350, 12681, 5296, 7353, 6616, 1180, 7486, 1061, 6114, 11107, 8648, 9917, 9720, 1052, 954, 6743, 7602, 6580, 9279, 3736, 3188, 1497, 5394, 3647, 2777, 3398, 2229, 6079, 9851, 10957, 8831, 3565, 5150, 1536, 12980, 9923, 3064, 66, 5979, 4855, 4345, 4406, 3720, 11869, 12606, 12064, 436, 10347, 13116, 13053, 1532, 746, 13129, 546, 2119, 8458, 13075, 6163, 10100, 5273, 1401, 13264, 7916, 12612, 5803, 11428, 6426, 9490, 4210, 6896, 5398, 3226, 4325, 9534, 11857, 13061, 7138, 10821, 9662, 4549, 5560, 171, 478, 8257, 5864, 7263, 10668, 2136, 2644, 11052, 9520, 2273, 3619, 5567, 9170, 10098, 3224, 13034, 12197, 2979, 3872, 370, 5242, 11026, 13458, 2972, 406, 8262, 9558, 6189, 6285, 13290, 643, 12652, 3144, 9077, 12018, 4328, 9, 1778, 8986, 12347, 13582, 9071, 2786, 10567, 4768, 1404, 8491, 6301, 4534, 4019, 5311, 5498, 659, 380, 6656, 10323, 4834, 10072, 4585, 4970, 2084, 2731, 9816, 1792, 12517, 7170

In [ ]:
PATH = '../data/components/features'
NETWORKS = ['coexpression', 
            'GO',
            'domain_sim',
            'sequence_sim',
            'pathway_co_occurrence']
NETWORK_PATH = '../data/components/networks/'
LABEL_PATH = '../data/components/labels/NIHMS80906-small_mol-and-bio-druggable.tsv'
PPIS = ['CPDB']


for ppi in PPIS:
    graph = load_h5_graph_shuffle(PATH, LABEL_PATH, ppi, shuffle_features=True)
    graph = add_all_edges(graph, NETWORK_PATH, NETWORKS)
    print(f"Number of nodes: {graph.num_nodes}")
    print(f"Number of edges: {graph.num_edges}")
    print(f"Number of features: {graph.num_features}")
    print(f"Number of edge types: {graph.num_edge_types}")
    torch.save(graph, f'../data/components/multidim_graph/6d/{ppi}_cdgps_shuffled_multiomics.pt')

Negative mask indices: [8274, 1696, 5846, 3422, 13287, 570, 5477, 12486, 9643, 515, 6859, 862, 5350, 12681, 5296, 7353, 6616, 1180, 7486, 1061, 6114, 11107, 8648, 9917, 9720, 1052, 954, 6743, 7602, 6580, 9279, 3736, 3188, 1497, 5394, 3647, 2777, 3398, 2229, 6079, 9851, 10957, 8831, 3565, 5150, 1536, 12980, 9923, 3064, 66, 5979, 4855, 4345, 4406, 3720, 11869, 12606, 12064, 436, 10347, 13116, 13053, 1532, 746, 13129, 546, 2119, 8458, 13075, 6163, 10100, 5273, 1401, 13264, 7916, 12612, 5803, 11428, 6426, 9490, 4210, 6896, 5398, 3226, 4325, 9534, 11857, 13061, 7138, 10821, 9662, 4549, 5560, 171, 478, 8257, 5864, 7263, 10668, 2136, 2644, 11052, 9520, 2273, 3619, 5567, 9170, 10098, 3224, 13034, 12197, 2979, 3872, 370, 5242, 11026, 13458, 2972, 406, 8262, 9558, 6189, 6285, 13290, 643, 12652, 3144, 9077, 12018, 4328, 9, 1778, 8986, 12347, 13582, 9071, 2786, 10567, 4768, 1404, 8491, 6301, 4534, 4019, 5311, 5498, 659, 380, 6656, 10323, 4834, 10072, 4585, 4970, 2084, 2731, 9816, 1792, 12517, 7170

In [ ]:
PATH = '../data/components/features'

f = h5py.File(f'{PATH}/CPDB_multiomics.h5', 'r')

# Assume 'features' is loaded from the HDF5 file
original_features = f['features'][:]  # Original feature matrix


np.random.seed(42)
# Shuffle rows
row_perm = np.random.permutation(original_features.shape[0])
shuffled_features = original_features[row_perm, :]
# Shuffle columns
col_perm = np.random.permutation(shuffled_features.shape[1])
shuffled_features = shuffled_features[:, col_perm]

# Check if the shuffled features differ from the original
if np.array_equal(original_features, shuffled_features):
    print("Randomisation did not occur!")
else:
    print("Randomisation worked.")

Randomisation worked.


In [168]:
original_features

array([[0.        , 0.00528242, 0.        , ..., 0.        , 0.44933921,
        0.        ],
       [0.        , 0.00530162, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06334503, 0.01576535, 0.05567673, ..., 0.19026549, 0.        ,
        0.        ],
       ...,
       [0.06282843, 0.06191132, 0.10887994, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00529114, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01060755, 0.        , ..., 0.        , 0.        ,
        0.04020101]])

In [169]:
shuffled_features

array([[5.96476079e-02, 2.19295631e-01, 1.97262425e-01, ...,
        8.09433519e-02, 0.00000000e+00, 0.00000000e+00],
       [1.16183458e-02, 2.25615841e-01, 0.00000000e+00, ...,
        6.28503206e-02, 4.44444444e-02, 0.00000000e+00],
       [1.53140960e-02, 1.56556732e-02, 1.71821051e-02, ...,
        6.90511697e-05, 0.00000000e+00, 0.00000000e+00],
       ...,
       [7.71920226e-03, 1.05705888e-02, 2.48456866e-02, ...,
        2.21097236e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 7.22422466e-04, 5.97364331e-03, ...,
        1.43317822e-02, 0.00000000e+00, 0.00000000e+00],
       [2.19949291e-02, 1.44105703e-02, 1.08093470e-02, ...,
        1.11558513e-02, 0.00000000e+00, 0.00000000e+00]])